#### Q1. You are working on a machine learning project where you have a dataset containing numerical and  categorical features. You have identified that some of the features are highly correlated and there are  missing values in some of the columns. You want to build a pipeline that automates the feature  engineering process and handles the missing values.

Design a pipeline that includes the following steps: 

Use an automated feature selection method to identify the important features in the dataset 

Create a numerical pipeline that includes the following steps:  

Impute the missing values in the numerical columns using the mean of the column values.
 
Scale the numerical columns using standardisation.
 
Create a categorical pipeline that includes the following steps:
 
Impute the missing values in the categorical columns using the most frequent value of the column.
 
One-hot encode the categorical columns.
 
Combine the numerical and categorical pipelines using a ColumnTransformer.
 
Use a Random Forest Classifier to build the final model.
 
Evaluate the accuracy of the model on the test dataset.

Note: Your solution should include code snippets for each step of the pipeline, and a brief explanation of  each step. You should also provide an interpretation of the results and suggest possible improvements for  the pipeline.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score


In [5]:
# Load the dataset
data = pd.read_csv('synthetic_data.csv')

# Display the first few rows of the dataset
data.head()


,feature1,feature2,feature3,target,category_feature
0,37.454012,1.571459,19.260949,0,B
1,95.071431,NaN,2.524199,1,A
2,73.199394,15.717799,4.848861,1,B
3,59.865848,25.428535,26.956626,1,B
4,NaN,45.378324,18.192872,0,B


In [4]:
data.tail()

,feature1,feature2,feature3,target,category_feature
95,49.379560,17.460479,15.667298,1,C
96,52.273283,NaN,23.099807,0,B
97,NaN,44.855513,6.474631,1,A
98,2.541913,NaN,18.686714,1,B
99,10.789143,38.993777,2.560424,0,A


In [6]:
# Define features and target
X = data.drop('target', axis=1)
y = data['target']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Define numerical and categorical features
numerical_features = ['feature1', 'feature2', 'feature3']
categorical_features = ['category_feature']

# Numerical pipeline
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
    ('scaler', StandardScaler())  # Standardize numerical features
])

# Categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])


In [8]:
# Combine numerical and categorical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])


In [9]:
# Final pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))  # Use Random Forest Classifier
])


In [10]:
# Fit the model
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['feature1', 'feature2',
                                                   'feature3']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['category_feature'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [11]:
# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.45


### Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then  use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its  accuracy.



In [14]:
# Import necessary libraries
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression  # Correct import
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Load the iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the individual classifiers
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
lr_classifier = LogisticRegression(max_iter=200, random_state=42)

# Create a voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('rf', rf_classifier),
        ('lr', lr_classifier)
    ],
    voting='hard'  # Use 'soft' for probability-based voting
)

# Create a pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scaling the features
    ('voting', voting_classifier)
])

# Train the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of Voting Classifier: {accuracy:.2f}')


Accuracy of Voting Classifier: 1.00
